In [2]:
from dataloader import *
%matplotlib inline
import sys
sys.path.append("./coco-caption")
import matplotlib.pyplot as plt
import skimage.io as io
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)
from json import encoder
encoder.FLOAT_REPR = lambda o: format(o, '.3f')
from dataloader import get_dataset_configuration, load_all_captions_flickr, load_all_captions_coco
import glob
from IPython.display import display, clear_output
from collections import deque
from ipywidgets import HBox, Output, Button
from prettytable import PrettyTable

In [3]:
import json
import os
import csv
from config import general
intersection=dict()
max_set_of_ids=set()
max_dict_of_ids=list()
min_dict_of_ids=list()
min_set_of_ids=set()

bleu1_ids=list()
bleu2_ids=list()
bleu3_ids=list()
bleu4_ids=list()
cider_ids=list()
meteor_ids=list()
rouge_ids=list()
for result_config_name in os.listdir(general["results_directory"]):
    if result_config_name.endswith(".json"):
        results = json.load(open("./" + general["results_directory"] + "/" + result_config_name, 'r'))
        dataset_name=results["dataset_name"]
        result_images_ids = list(results['imgToEval'].keys())
        bleu1=dict()
        bleu2=dict()
        bleu3=dict()
        bleu4=dict()
        rouge=dict()
        meteor=dict()
        cider=dict()
        for img_id in result_images_ids:
            bleu1[img_id]=results['imgToEval'][img_id]['Bleu_1']
            bleu2[img_id]=results['imgToEval'][img_id]['Bleu_2']
            bleu3[img_id]=results['imgToEval'][img_id]['Bleu_3']
            bleu4[img_id]=results['imgToEval'][img_id]['Bleu_4']
            rouge[img_id]=results['imgToEval'][img_id]['ROUGE_L']
            meteor[img_id]=results['imgToEval'][img_id]['METEOR']
            cider[img_id]=results['imgToEval'][img_id]['CIDEr']

        def sort_and_get_max_min(metric, name):
            sorted_by=sorted(metric.items(), key=lambda x:x[1])
            sorted_max_n=sorted_by[1]
            image_id_max=sorted_max_n[0]
            image_id_min=sorted_max_n[0]
            min_set_of_ids.append(image_id_min)
            if image_id_min not in min_dict_of_ids:
                if min_dict_of_ids[image_id_min]["measures"]:
                    a=min_dict_of_ids[image_id_min]["measures"]
                    b=a.append(name)
                    min_dict_of_ids[image_id_min]["measures"]=b
                else:
                    min_dict_of_ids[image_id_min]={"measures":[name]}
            max_set_of_ids.append(image_id_max)
            if image_id_max not in max_dict_of_ids:
                if max_dict_of_ids[image_id_min]["measures"]:
                    a=max_dict_of_ids[image_id_min]["measures"]
                    b=a.append(name)
                    max_dict_of_ids[image_id_max]["measures"]=b
                else:
                    max_dict_of_ids[image_id_max]={"measures":[name]}
            return image_id_max,image_id_min, sorted_by

        bleu1_sorted_max_n, bleu1_sorted_min_n, bleu1_sorted=sort_and_get_max_min(bleu1)
        bleu1_ids.append(bleu1_sorted_max_n)
        bleu1_ids.append(bleu1_sorted_min_n)
        bleu2_sorted_max_n, bleu2_sorted_min_n, bleu2_sorted=sort_and_get_max_min(bleu2)
        bleu2_ids.append(bleu2_sorted_max_n)
        bleu2_ids.append(bleu2_sorted_min_n)
        bleu3_sorted_max_n, bleu3_sorted_min_n, bleu3_sorted=sort_and_get_max_min(bleu3)
        bleu3_ids.append(bleu3_sorted_max_n)
        bleu3_ids.append(bleu3_sorted_min_n)
        bleu4_sorted_max_n, bleu4_sorted_min_n, bleu4_sorted=sort_and_get_max_min(bleu4)
        bleu4_ids.append(bleu4_sorted_max_n)
        bleu1_ids.append(bleu4_sorted_min_n)
        meteor_sorted_max_n, meteor_sorted_min_n, meteor_sorted=sort_and_get_max_min(meteor)
        meteor_ids.append(meteor_sorted_max_n)
        meteor_ids.append(meteor_sorted_min_n)
        rouge_sorted_max_n, rouge_sorted_min_n, rouge_sorted=sort_and_get_max_min(rouge)
        rouge_ids.append(rouge_sorted_max_n)
        rouge_ids.append(rouge_sorted_min_n)
        cider_sorted_max_n, cider_sorted_min_n, cider_sorted=sort_and_get_max_min(cider)
        cider_ids.append(cider_sorted_max_n)
        cider_ids.append(cider_sorted_min_n)

        intersection_max={'Bleu_1':bleu1_sorted_max_n,'Bleu_2':bleu2_sorted_max_n, 'Bleu_3':bleu3_sorted_max_n,
                          'Bleu_4':bleu4_sorted_max_n, 'ROUGE_L':rouge_sorted_max_n, 'METEOR':meteor_sorted_max_n,
                          'CIDEr':cider_sorted_max_n}
        intersection_min={'Bleu_1':bleu1_sorted_min_n,'Bleu_2':bleu2_sorted_min_n, 'Bleu_3':bleu3_sorted_min_n,
                          'Bleu_4':bleu4_sorted_min_n, 'ROUGE_L':rouge_sorted_min_n, 'METEOR':meteor_sorted_min_n,
                          'CIDEr':cider_sorted_min_n}

        print(intersection_max)
        print(len(intersection_min))
        print("...........................")
        intersection[result_config_name]={"max_n":intersection_max, "min_n":intersection_min, "dataset_name":dataset_name}

        # image_results = info['imgToEval'][image_id]
        # show_image_results_captions(image_id)

with open("./" + general["results_directory"] +"/intersection"+ "/intersection_results.json", 'w') as outfile:
    json.dump(
        {'intersection': intersection, "max_list_of_ids":max_list_of_ids, "min_list_of_ids":min_list_of_ids,"bleu1_ids":bleu1_ids,
         "bleu2_ids":bleu2_ids,
         "bleu3_ids":bleu3_ids,
         "bleu4_ids":bleu4_ids,
         "cider_ids":cider_ids,
         "meteor_ids":meteor_ids,
         "rouge_ids":rouge_ids},outfile)

In [ ]:
def load_images_flickr(images_dir):
    """
    Method to map images ids to pictures for data in Flickr structure

    Parameters
    ----------
    images_dir: str
        Path to the directory with all images from  Flickr type dataset
    Returns
    -------
    all_images_mapping: dict->
        paths to all images

    """
    # add global paths to the all images in images_dir directory
    all_images = glob.glob(images_dir + '*.jpg')
    all_images_mapping = dict()
    for i in all_images:  # img is list of full path names of all images
        image_name = i.split("/")[-1]
        image_id = image_name.split(".")[0]
        all_images_mapping[image_id] = i  # Add it to the dict of train images
    return all_images_mapping

def load_images_coco(configuration):
    """
    Method to map images ids to pictures for data in COCO structure

    Parameters
    ----------
    configuration
        Configuration of the dataset, with paths to the images and
         datasets specific files like file that mapps images with image id
    Returns
    -------
    all_images_mapping: dict->
        paths to all images from coco type data identidied by image ids

    """
    file_with_images_def = configuration["images_names_file_path"]
    images_folder = configuration["images_dir"]
    info = json.load(open(file_with_images_def))
    all_images_mapping = dict()
    for ix in range(len(info['images'])):
        img = info['images'][ix]
        image_filename = img['file_path'].rsplit(".", 1)[0]
        #create global path to the image by users directory
        file_path = images_folder + "/" + img['file_path']

        if image_filename.find("/") != -1:
            image_filename = img['file_path'].rsplit("/", 1)[1].rsplit(".", 1)[0]
        #define data splits
        if img['split'] in ['train','val', 'test', 'restval']:
            all_images_mapping[image_filename] = file_path

    return all_images_mapping

def get_images_for_split(dataset_name):
    # Load dataset configuration, by the name of the dataset assigned for training/testing
    train_dataset_configuration = get_dataset_configuration(dataset_name)
    # Therefore Flickr and COCO have different file and data structures, to show captions and split of data
    # different methods for loading captions and images are used.
    # Datasets Flickr30k, COCO2017, COCO2014 have the same strucutre of files with captions and split informations.
    if train_dataset_configuration["data_name"] in ["flickr30k", "coco17", "coco14"]:
        all_images = load_images_coco(train_dataset_configuration)
    # Datasets Flickr30k, Flickr8k_polish, AIDe, Flickr8k  have the same strucutre of files with captions and split informations.
    if train_dataset_configuration["data_name"] in ["flickr30k_polish", "flickr8k_polish", "aide", "flickr8k"]:
        all_images = load_images_flickr(train_dataset_configuration["images_dir"])
    return all_images


In [4]:

def show_image_results_captions(image_id, dataset_name, result_config_name):
    """
    Method to show image, ground truth captions, predicted caption and results of metrics
    Parameters
    ----------
    image_id: str
        ID of image
    Returns
    -------
    Prints image, ground truth captions, predicted caption and results of metrics
    """
    all_images_from_split = get_images_for_split(dataset_name)
    #Load results of metrics from file
    results_info = json.load(open("./" + general["results_directory"] + "/" + result_config_name, 'r'))
    image_results = results_info['imgToEval'][image_id]
    print('Dataset name: {}'.format(dataset_name))
    #Load image
    I = io.imread(all_images_from_split[image_id])
    plt.imshow(I)
    plt.axis('off')
    plt.show()
    print("Ground truth captions")
    print(image_results['ground_truth_captions'])
    print("Predicted captions")
    print(image_results['caption'])
    #Display results in pretty table
    print( f'\n===== Results =====' )
    t = PrettyTable(("Bleu_1", "Bleu_2", "Bleu_3", "Bleu_4"))
    t.add_row((image_results["Bleu_1"], image_results["Bleu_2"], image_results["Bleu_3"], image_results["Bleu_4"]))
    t2 = PrettyTable(("METEOR", "ROUGE_L", "CIDEr", "WMD"))
    t2.add_row((image_results["METEOR"],image_results["ROUGE_L"], image_results["CIDEr"], image_results["WMD"]))
    print(t)
    print(t2)
    print()

intersection_results = json.load(open("./" + general["results_directory"] + "/intersection_results.json"))
config_names=list(intersection_results['intersection'].keys())
type_of="max_n"
for config in config_names:
    b=config[type_of]["BLeu_4"]
    dataset_name=config[dataset_name]
    show_image_results_captions(b, dataset_name, config)
    m=config[type_of]["METEOR"]
    r=config[type_of]["ROUGE_L"]
    c=config[type_of]["CIDEr"]

['mixed_flickr8k_8k_n.json', 'mixed_coco2014_coco2014.json']


Select(description='Name of the dataset:', options=('mixed_flickr8k_8k_n.json', 'mixed_coco2014_coco2014.json'…

In [5]:
#Create fancy viever for images, captions and results of evaluation
d=deque(images_ids)
#Button to read image back
left = Button(description="<")
#Button to read next image
right = Button(description=">")

switch = [left, right]

combined = HBox([items for items in switch])
out = Output()

def on_button_left(ex):
    with out:
        clear_output()
        d.rotate(1)
        show_image_results_captions(d[0])
def on_button_right(ex):
    with out:
        clear_output()
        d.rotate(-1)
        show_image_results_captions(d[0])
l=switch[0].on_click(on_button_left)
r=switch[1].on_click(on_button_right)
display(combined)
display(out)